### Init Context

In [4]:
from thetaray.api.context import init_context
from datetime import datetime
import yaml
import pyspark.sql.functions as f

import logging
logging.basicConfig(level=logging.DEBUG, format='%(message)s')

with open('/thetaray/git/solutions/domains/demo_merchant/config/spark_config.yaml') as spark_config_file:
    spark_config = yaml.load(spark_config_file, yaml.FullLoader)['spark_config_a']

context = init_context(
    execution_date=datetime(1970, 1, 1),
    spark_conf=spark_config, 
    # spark_master='local[*]', # drop
)


/thetaray/venv/lib64/python3.11/site-packages/starlette/config.py:60: UserWarning: Config file '.env' not found.
  warnings.warn(f"Config file '{env_file}' not found.")
2025-08-21 14:41:21,664:INFO:thetaray.common.logging:start loading solution.....[ load_risks=True , solution_path=/thetaray/git/solutions/domains , settings_path=/thetaray/git/solutions/settings ]
2025-08-21 14:41:21,673:WARNING:thetaray.common.logging:Encryption is enabled on dataset demo_human_trafficking_insights, but data encryption is disabled in deployment
2025-08-21 14:41:21,689:WARNING:thetaray.common.logging:Encryption is enabled on dataset demo_remittance_customer_insights, but data encryption is disabled in deployment
2025-08-21 14:41:21,699:WARNING:thetaray.common.logging:Encryption is enabled on dataset demo_ret_smb_customer_insights, but data encryption is disabled in deployment
2025-08-21 14:41:21,737:WARNING:thetaray.common.logging:Encryption is enabled on dataset wrangling, but data encryption is disabl

Added `alias` successfully.


2025-08-21 14:42:05,180:INFO:thetaray.common.logging:Creating metastore schema for evaluation flow: demo_merchant_ef
2025-08-21 14:42:05,342:INFO:thetaray.common.logging:=== Finished updating schema for Evaluation Flows on Minio ===


Added `alias` successfully.


### Params

In [5]:
from thetaray.api.dataset import dataset_functions
from thetaray.api.evaluation import evaluate, write_evaluated_activities, publish_evaluated_activities, load_evaluated_activities
from thetaray.common.data_environment import DataEnvironment

from domains.demo_merchant.datasets.customer_monthly import customer_monthly_dataset as dataset
from domains.demo_merchant.evaluation_flows.ef import evaluation_flow as ef

### Detection

In [6]:
data = dataset_functions.read(context, 
                              dataset().identifier,
                              data_environment=DataEnvironment.PUBLIC).drop('tr_timestamp')

In [7]:
print("Number of transactions in 3_detection:", data.count())

[Stage 16:======================================>                   (2 + 1) / 3]

Number of transactions in 3_detection: 2700


In [9]:
evaluated_data = evaluate(context, 
                          ef().identifier, 
                          data,
                          data_environment=DataEnvironment.PUBLIC)

2025-08-21 14:43:43,239:INFO:thetaray.common.logging:Applying backwards compatibility patches
2025-08-21 14:43:43.993683: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-08-21 14:43:43.995691: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-08-21 14:43:43.999334: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-08-21 14:43:44.007464: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1755787424.020965   14582 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to re

2025-08-21 14:43:46,683:INFO:thetaray.common.logging:Rolling back backwards compatibility patches
2025-08-21 14:43:46,684:INFO:thetaray.common.logging:Rolled back backwards compatibility patches
2025-08-21 14:43:46,684:INFO:thetaray.common.logging:Applying backwards compatibility patches
2025-08-21 14:43:46,689:INFO:thetaray.common.logging:Applied backwards compatibility patches


2025-08-21 14:43:47,064:INFO:thetaray.common.logging:Rolling back backwards compatibility patches
2025-08-21 14:43:47,065:INFO:thetaray.common.logging:Rolled back backwards compatibility patches


In [15]:
print('Number of alerts with algo_score > .5:', evaluated_data.filter(f.col('algo_score')>.5).count())

[Stage 24:======================================>                   (2 + 1) / 3]

Number of alerts with algo_score > .5: 54


In [16]:
# Total de clientes por mes
df_clients_month = evaluated_data.groupBy('year_month') \
    .count() \
    .orderBy('year_month') \
    .toPandas()

# Total de alertas por mes
df_alerts_months = evaluated_data.filter(f.col('algo_score') >= 0.5) \
    .groupBy('year_month') \
    .count() \
    .withColumnRenamed('count', 'count_alerts') \
    .orderBy('year_month') \
    .toPandas()

# Merge y cálculo de porcentaje de alertados
df_clients_alerts = df_clients_month.merge(df_alerts_months, how='left', on='year_month')
df_clients_alerts['count_alerts'] = df_clients_alerts['count_alerts'].fillna(0)
df_clients_alerts['pct_alerted'] = 100 * df_clients_alerts['count_alerts'] / df_clients_alerts['count']

# Mostrar
df_clients_alerts

,year_month,count,count_alerts,pct_alerted
0,2024-01,150,0.0,0.000000
1,2024-02,150,0.0,0.000000
2,2024-03,150,0.0,0.000000
3,2024-04,150,19.0,12.666667
4,2024-05,150,1.0,0.666667
5,2024-06,150,1.0,0.666667
6,2024-07,150,14.0,9.333333
7,2024-08,150,0.0,0.000000
8,2024-09,150,0.0,0.000000
9,2024-10,150,3.0,2.000000


In [ ]:
from thetaray.api.evaluation import unblock_evaluation_for_execution_date


unblock_evaluation_for_execution_date(
    context,
    evaluation_flow_identifier=ef().identifier,
    data_environment=DataEnvironment.PUBLIC)

In [ ]:
write_evaluated_activities(context, 
                           evaluated_data, 
                           ef().identifier,
                           data_environment=DataEnvironment.PUBLIC)

In [ ]:
publish_evaluated_activities(context, 
                             ef().identifier,
                             data_environment=DataEnvironment.PUBLIC)

In [ ]:
context.close()